# Cartoonizer:

# 1. Cartoonizing an Image 

This project converts a live picture into a cartoon-like image. Either it is
a B/W sketch or a colored cartoon image


There are only two libraries used in the project:

1. openCV( Open source computer vision ): used for videocapturing and rendering
2. numpy: used for numerical computation


In [1]:
# importing libraries
import cv2
import numpy as np

# Cartoonize function

def cartoonize_image(img , ds_factor=4,sketch_mode=False):
    
    # Converting the image to grey scale
    # cvtColor() : built in function in cv2 which one color to other
    # medianBlur() : computes median of all the pixels in kernal window and replaces the central pixel with it
    # Laplacian() : used to implement dicrete analog of laplace operator
    
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_gray = cv2.medianBlur(img_gray, 7)
    edges = cv2.Laplacian(img_gray,cv2.CV_8U, ksize=5)
    
    #ret: boolean value returned by read() function it indicates whether the frame is captured successfully or not
    
    ret, mask = cv2.threshold(edges, 100,255, cv2.THRESH_BINARY_INV)
    if sketch_mode:
        return cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    img_small = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    # resize() :  used for scaling image
    
    img_small = cv2.resize(img, None,fx=1.0/ds_factor, fy=1.0/ds_factor,interpolation=cv2.INTER_AREA)
    
    num_repetitions = 10
    sigma_color = 5
    sigma_space = 7
    size = 5
    for i in range(num_repetitions):
        img_small=cv2.bilateralFilter(img_small, size,sigma_color, sigma_space)
        
    img_output = cv2.resize(img_small,None, fx=ds_factor, fy=ds_factor,interpolation=cv2.INTER_LINEAR)
    dst = np.zeros(img_gray.shape)
    dst = cv2.bitwise_and(img_output,img_output, mask=mask)
    return dst

In [2]:
if __name__ == "__main__":
    
    # VideoCaputre(): used for capturing images and adding effects to image
    
    cap = cv2.VideoCapture(0)
    cur_char = -1
    prev_char = -1
    while True:
        ret, frame = cap.read()
        frame = cv2.resize(frame, None, fx=0.5 , fy=0.5,interpolation=cv2.INTER_AREA)
        c=cv2.waitKey(1)
        
        # ASCII value of ESC is 27
        if c== 27:
            break
        
        if c > -1 and c != prev_char:
            cur_char = c
        prev_char = c
        
        if cur_char == ord('s'):
            cv2.imshow('Cartoonize',cartoonize_image(frame, sketch_mode=True))
        elif cur_char == ord('c'):
            cv2.imshow('Cartoonize',cartoonize_image(frame,sketch_mode=False))
        else:
            cv2.imshow('Cartoonize', frame)
            
    cap.release()
    cv2.destroyAllWindows()